In [45]:
import pandas as pd
import seaborn as sns
import openpyxl
import math
import re
import numpy as np

In [148]:
zipdata = pd.read_csv("ExternalDataByZipcode.csv")

# convert data into usuable types
zipdata['Median Household Income'] = zipdata['Median Household Income'].str.replace('\W', '', regex=True).astype(float)
zipdata['Population'] = zipdata['Population'].str.replace('\W', '', regex=True).astype(float)
zipdata['White Percentage'] = (zipdata['White Percentage'].str.replace('\W', '', regex=True).astype(float)) / 100

# print(zipdata.var())
# print(zipdata.dtypes)
zipdata.head()

# for each zip code find standardized euclidean norm (2-norm) of varialbes
# vars = [income, age, population, nonwhite]

# initialize list of norms
norm = []

# select which zipcode to test
zipcode = 77026
zipindex = zipdata[zipdata['Zip Codesort column'] == zipcode].index.values
targeted = [zipindex[0]]

for precinct in targeted:
    # retrieves values of variables for zipcodes of targeted precincts
    tar_income = zipdata.iloc[precinct]['Median Household Income']
    tar_age = zipdata.iloc[precinct]['Median Age']
    tar_pop = zipdata.iloc[precinct]['Population']
    tar_div = 100 - zipdata.iloc[precinct]['White Percentage']

    for i in range(zipdata.shape[0]):
        # takes out testing precinct with itself
        if precinct != i:
            # retrieves values of variables for other zipcodes
            income = zipdata.iloc[i]['Median Household Income']
            age = zipdata.iloc[i]['Median Age']
            pop = zipdata.iloc[i]['Population']
            div = 100 - zipdata.iloc[i]['White Percentage']
            # standardizes the difference between the targeted and other zipcodes
            # i.e. income and age are varied across vastly different scales
            incomediff = (tar_income - income) / (1.067543e+09)
            agediff = (tar_age - age) / (1.693905e+01)
            popdiff = (tar_pop - pop) / (1.558452e+08)
            divdiff = (tar_div - div) / (3.090548e+02)
            # sums the standardized differences
            sum = ((incomediff)**2) + ((agediff)**2) + ((popdiff)**2) + ((divdiff)**2)
        # takes the square root of the sum according to formula
        norm.append(math.sqrt(sum))

# print(norm)
print('standardized euclidean norm:', min(norm))
indexnorm = np.argmin(norm)
print('targeted zipcode:', zipdata.loc[[targeted[0]]])
print('closest other zipcode:', zipdata.loc[[indexnorm]])

standardized euclidean norm: 0.012032018430247414
targeted zipcode:     Zip Codesort column  Median Household Income  Median Age  Population  \
24                77026                  32972.0        34.8      5582.0   

    White Percentage Black/AfricanAmerican Percentage  \
24             24.08                           47.96%   

   Pacific Islander Percentage Asian Percentage Other Race Population  \
24                       0.04%            0.43%                23.18%   

   2+ Races Percentage     sum  
24               3.71%  99.40%  
closest other zipcode:     Zip Codesort column  Median Household Income  Median Age  Population  \
80                77083                  63264.0        34.9     22459.0   

    White Percentage Black/AfricanAmerican Percentage  \
80             27.32                           28.64%   

   Pacific Islander Percentage Asian Percentage Other Race Population  \
80                       0.03%           23.69%                16.40%   

   2+ Races P

In [149]:
import plotly.graph_objects as go

categories = ['median household income','median age','total population',
              'underrepresented voter percentage']

fig = go.Figure()